# Calculate Number of Frost Days

This notebook computes the annual number of frost days for a given year, by using the daily minimum temperature (`tasmin`). The number of frost days index is the annual count of days where `tasmin` < 0 °C.

This Jupyter notebook is meant to run in the Jupyterhub server of the German Climate Computing Center [DKRZ](https://www.dkrz.de/) which is an [ESGF](https://esgf.llnl.gov/) repository that hosts 4 petabytes of CMIP6 data. Please, choose the Python 3 unstable kernel on the Kernel tab above, it contains all the common geoscience packages. See more information on how to run Jupyter notebooks at DKRZ [here](https://www.dkrz.de/up/systems/mistral/programming/jupyter-notebook). Find there how to run this Jupyter notebook in the DKRZ server out of the Jupyterhub, which will entail that you create the environment accounting for the required package dependencies. Running this Jupyter notebook in your premise, which is also known as [client-side](https://en.wikipedia.org/wiki/Client-side) computing, will also require that you install the necessary packages on you own but it will anyway fail because you will not have direct access to the data pool. Direct access to the data pool is one of the main benefits of the [server-side](https://en.wikipedia.org/wiki/Server-side) data-near computing we demonstrate in this use case.

In this use case you will learn the following:
- How to access a dataset from the DKRZ CMIP6 model data archive
- How to count the annual number of frost days globally for a specified year
- How to visualize the results


You will use:
- [Intake](https://github.com/intake/intake) for finding the data in the catalog of the DKRZ archive
- [Xarray](http://xarray.pydata.org/en/stable/) for loading and processing the data
- [Cartopy](https://pypi.org/project/Cartopy/) for visualizing the data in the Jupyter notebook and save the plots in your local computer

## 0. Import Packages

In [ ]:
import intake           # to find data in a catalog, this notebook explains how it works
import xarray as xr     # handling labelled multi-dimensional arrays

# plotting libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

## 1. Select the Year for Frost Days Calculation

In [ ]:
year = "2100"

## 2. Intake Catalog
Similar to the shopping catalog at your favorite online bookstore, the intake catalog contains information (e.g. model, variables, and time range) about each dataset (the title, author, and number of pages of the book, for instance) that you can access before loading the data. It means that thanks to the catalog, you can find where is the book just by using some keywords and you do not need to hold it in your hand to know the number of pages, for instance.

### 2.1. Load the Intake Catalog and Orientate
We load the catalog descriptor with the intake package. The catalog is updated daily. The catalog descriptor is created by the DKRZ developers that manage the catalog, you do not need to care so much about it, knowing where it is and loading it is enough:

In [ ]:
# Path to catalog descriptor on the DKRZ server
col_url = "/pool/data/Catalogs/mistral-cmip6.json"

# Open the catalog with the intake package and name it "col" as short for "collection"
col = intake.open_esm_datastore(col_url)
col

Let's see what is inside the intake catalog. The underlying data base is given as a pandas dataframe which we can access with `col.df`. Hence, `col.df.head()` shows us the first rows of the table of the catalog.

In [ ]:
col.df.head()

This catalog contains all datasets of the CMIP6 archive at DKRZ. Before searching for the needed data file we will have a closer look at the cataloge.

100 climte models `source_id` are part of the catalog.

In [ ]:
col.unique(["source_id"])

The catalog offers 1162 variables.

In [ ]:
col.unique(["variable_id"])

You can create a subset of the catalog with pandas operations. Here we select all CMIP activities (`col.df["activity_id"] == "CMIP"`)

In [ ]:
col.df[col.df["activity_id"] == "CMIP"]

### 2.2. Browse the Intake Catalog

The most elegant way for creating subsets is a query with a dictionary. In our case we design the search dictionary as follows: We chose the Max-Planck Earth System Model in Low Resolution Mode ("MPI-ESM1-2-LR") and the minimm temperature near surface (`tasmin`) as variable. We also choose an experiment. CMIP6 comprises several kind of experiments. Each experiment has various simulation members. you can find more information in the CMIP6 Model and Experiment Documentation.

In [ ]:
# This is how we tell intake what data we want

query = dict(
    source_id      = "MPI-ESM1-2-LR", # here we choose Max-Plack Institute's Earth Sytem Model in high resolution
    variable_id    = "tasmin",        # temperature at surface, minimum
    table_id       = "day",           # daily frequency
    experiment_id  = "ssp585",        # what we selected in the drop down menu,e.g. SSP2.4-5 2015-2100
    member_id      = "r10i1p1f1",     # "r" realization, "i" initialization, "p" physics, "f" forcing
)

# Intake looks for the query we just defined in the catalog of the CMIP6 data pool at DKRZ
cat = col.search(**query)

del col # Make space for other python objects

# Show query results
cat.df

The result of the query are like the list of results you get when you search for articles in the internet by writing  keywords in your search engine (Duck duck go, Ecosia, Google,...). Thanks to the intake package, we did not need to know the path of each dataset, just selecting some keywords (the model name, the variable,...) was enough to obtain the results. If advance users are still interested in the location of the data inside the DKRZ archive, intake also provides the path and the OpenDAP URL (see the last columns above). 


Now we will find which file in the dataset contains our selected year so in the next section we can just load that specific file and not the whole dataset.

### 2.3. Find the Dataset

In [ ]:
# Create a copy of cat.df, thus further modifications do not affect it 
query_result_df = cat.df.copy() # new dataframe to play with

# Each dataset contains many files, extract the initial and final year of each file 
query_result_df["start_year"] = query_result_df["time_range"].str[0:4].astype(int) # add column with start year
query_result_df["end_year"] = query_result_df["time_range"].str[9:13].astype(int) # add column with end year

# Delete the time range column
query_result_df.drop(columns=["time_range"], inplace = True) # "inplace = False" will drop the column in the view but not in the actual dataframe
query_result_df.iloc[0:3]

# Select the file that contains the year we selected in the drop down menu above, e.g. 2015
selected_file = query_result_df[(int(year) >= query_result_df["start_year"]) & (
                   int(year) <= query_result_df["end_year"])]

# Path of the file that contains the selected year    
selected_path = selected_file["path"].values[0] 

# Show the path of the file that contains the selected year
selected_path

## 3. Load the Model Data

In [ ]:
ds = xr.open_dataset(selected_path)

# Open variable "tasmin" over the whole time range
ds_tasmin = ds["tasmin"]

Look at data

In [ ]:
ds_tasmin

Select year

In [ ]:
ds_tasmin_year = ds_tasmin.sel(time=year)

Count the number of frost days

In [ ]:
ds_tasmin_year_count = ds_tasmin_year.where(ds_tasmin_year < 273.15).count(dim='time')

Before plotting, a cyclic point has to be added, otherwise there will be a gap at the prime meridian.

In [ ]:
lon = ds_tasmin_year_count.lon
lat = ds_tasmin_year_count.lat
ds_tasmin_year_count, lon = add_cyclic_point(ds_tasmin_year_count, lon)

## 4. Plot data with `cartopy`

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

plt.contourf(lon, lat, ds_tasmin_year_count, 60,
            transform=ccrs.PlateCarree(),
            cmap='Blues')
ax.coastlines()
ax.set_global()

# Add a color bar
plt.colorbar(ax=ax)
plt.title('Number of Frost Days in Year ' +year)
plt.show()